# Database Explorer - Simple View

This notebook provides a simple overview of all tables in the database.

In [ ]:
import duckdb
import pandas as pd
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

DATABASE_PATH = Path('data/hospital_worksheets.duckdb')
print(f"Database: {DATABASE_PATH}")
print(f"Exists: {DATABASE_PATH.exists()}")

## 1. Tables Available

In [ ]:
con = duckdb.connect(str(DATABASE_PATH), read_only=True)

# Get all tables
tables_df = con.execute("""
    SELECT 
        table_name,
        table_type
    FROM information_schema.tables
    WHERE table_schema = 'main'
    ORDER BY table_type, table_name
""").df()

print(f"\nTotal tables and views: {len(tables_df)}\n")
display(tables_df)

## 2. Sample Data from Each Table (LIMIT 10)

In [ ]:
# Get list of base tables only (not views)
tables = con.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
        AND table_type = 'BASE TABLE'
    ORDER BY table_name
""").df()['table_name'].tolist()

print(f"Found {len(tables)} tables\n")

# Loop through each table and display first 10 rows
for table_name in tables:
    print("="*80)
    print(f"TABLE: {table_name}")
    print("="*80)
    
    # Get row count
    count = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    print(f"Total rows: {count:,}\n")
    
    # Get first 10 rows
    df = con.execute(f"SELECT * FROM {table_name} LIMIT 10").df()
    display(df)
    print("\n")

con.close()
print("✓ Done")